# ML regression to predict the Potency of an active inhibitor regarding G9a

### Content   <a name="content"></a>

1. [Data loading and observations](#1)
2. [Regression Machine Learning](#2)
3. [Cros-validation](#3)
4. [Feature importance](#4)
5. [Comparison between different feature importance methods. Creation of a model only with solubility data ](#5)
6. [Hyperparameter tuning of the model with the solubility data only](#6)
7. [Demo with samples unseen by the five feature model](#7)

### Data loading and observations <a name="1"></a>

In [1]:
 # pip install modin[ray] 

In [1]:
import pandas as pd 

# # loading the dataset for the regression ML
df = pd.read_csv('data_classification_smote.csv', index_col=[0])
# Avoid some columns to be truncated during df display
pd.set_option('display.max_columns', None)
# Display the data frame
print('Shape of df: ', df.shape)
df.head()

Shape of df:  (7742, 60)


,MW,TPSA,XL,HAC,HBDC,HBAC,RBC,CBUC,MMX6,MMX,SX6,SX,MMY6,MMY,SY6,SY,Volume_1,Volume_2,MMX6_3D,MMX_3D,SX6_3D,SX_3D,MMY6_3D,MMY_3D,SY6_3D,SY_3D,MMZ6_3D,MMZ_3D,SZ6_3D,SZ_3D,Volume_1_3D,XY_3D_volume,XZ_3D_volume,YZ_3D_volume,C_relative,H_relative,O_relative,S_relative,N_relative,Br_relative,Cl_relative,F_relative,C,H,O,S,N,Br,Cl,F,C_rel_2D,allAtoms_rel_2D,C_rel_XY_3D,allAtoms_rel_XY_3D,C_rel_XZ_3D,allAtoms_rel_XZ_3D,C_rel_YZ_3D,allAtoms_rel_YZ_3D,Similarity,target
0,331.4,77.7,21.5,24,0,6,6,1,10.0704,10.0704,20.449255,20.330695,12.6774,12.6774,19.953895,20.014980,0.000000,9.209391e-08,10.3833,12.2639,19.907586,20.239100,5.5975,7.1501,19.513505,19.637322,2.6922,4.7019,19.186260,19.675478,34.432971,20.008264,20.004219,20.005413,0.38,0.47,0.09,0.00,0.07,0.0,0.00,0.00,61.62,6.39,19.31,0.00,12.68,0.0,0.00,0.00,20.794358,20.794358,1.854989,1.715207,3.856809,2.608286,2.079155,1.520683,20.065,0
1,320.8,65.4,24.0,21,2,2,4,1,10.3831,11.4240,20.064455,20.116348,9.7610,9.7610,19.975572,19.986373,0.083246,1.734270e-04,8.5927,10.6824,19.780021,19.559252,5.3850,7.6866,19.811921,20.006070,2.9047,4.5766,20.253256,20.224332,64.661858,20.260906,20.212295,20.075279,0.42,0.45,0.03,0.03,0.05,0.0,0.03,0.00,59.90,5.34,4.99,9.99,8.73,0.0,11.05,0.00,21.063733,21.170372,1.595673,1.389743,2.958206,2.334135,1.853892,1.679544,20.053,0
2,354.4,66.9,24.7,27,2,4,4,1,11.4134,11.4134,20.294225,20.232029,9.7636,10.9636,20.216582,20.168453,0.195432,1.246694e-03,11.4574,13.1701,19.313819,19.564397,9.0595,11.2258,19.634972,19.799344,2.2779,4.0079,19.444561,19.623878,41.199505,20.029067,20.009599,20.037149,0.49,0.40,0.02,0.00,0.09,0.0,0.00,0.00,74.56,5.12,4.51,0.00,15.81,0.0,0.00,0.00,21.168975,21.041027,1.264683,1.173199,5.029808,3.286035,3.977128,2.800918,20.046,1
3,327.3,95.9,23.2,21,2,7,1,1,7.4095,8.6034,20.247273,19.981341,9.0468,10.1334,19.674979,20.313131,1.158458,7.001743e-03,8.6098,10.3467,20.480878,20.448428,3.6421,5.4166,20.249601,20.261903,2.5441,4.0777,19.653137,19.848343,22.342195,20.064190,20.024136,20.021322,0.32,0.38,0.06,0.03,0.09,0.0,0.00,0.12,40.37,4.00,9.78,9.80,12.84,0.0,0.00,23.22,20.819019,20.849014,2.363966,1.910184,3.384222,2.537386,1.431587,1.328347,20.074,1
4,317.3,74.6,23.3,24,2,4,4,1,11.6967,12.7360,20.306222,20.279696,6.7506,7.5002,19.753227,19.758681,0.289777,1.344280e-03,11.2244,14.0997,20.251410,20.227457,6.4817,8.6167,20.633351,20.394855,2.3768,4.2833,19.972298,19.829939,136.973233,20.927876,20.607823,20.401540,0.49,0.38,0.05,0.00,0.08,0.0,0.00,0.00,71.91,4.76,10.08,0.00,13.24,0.0,0.00,0.00,21.732690,21.698088,1.731706,1.636322,4.722484,3.291784,2.727070,2.011697,20.041,0


In [2]:
# Create a data set only with isomers that are inactive inhibitors 
df = df[df['target']==1]

# Display the data set with the isomers target 0 
print('Shape of df_isomers_target_1: ', df.shape)
df.head()

Shape of df_isomers_target_1:  (3871, 60)


,MW,TPSA,XL,HAC,HBDC,HBAC,RBC,CBUC,MMX6,MMX,SX6,SX,MMY6,MMY,SY6,SY,Volume_1,Volume_2,MMX6_3D,MMX_3D,SX6_3D,SX_3D,MMY6_3D,MMY_3D,SY6_3D,SY_3D,MMZ6_3D,MMZ_3D,SZ6_3D,SZ_3D,Volume_1_3D,XY_3D_volume,XZ_3D_volume,YZ_3D_volume,C_relative,H_relative,O_relative,S_relative,N_relative,Br_relative,Cl_relative,F_relative,C,H,O,S,N,Br,Cl,F,C_rel_2D,allAtoms_rel_2D,C_rel_XY_3D,allAtoms_rel_XY_3D,C_rel_XZ_3D,allAtoms_rel_XZ_3D,C_rel_YZ_3D,allAtoms_rel_YZ_3D,Similarity,target
2,354.40,66.9,24.7,27,2,4,4,1,11.4134,11.4134,20.294225,20.232029,9.7636,10.9636,20.216582,20.168453,0.195432,0.001247,11.4574,13.1701,19.313819,19.564397,9.0595,11.2258,19.634972,19.799344,2.2779,4.0079,19.444561,19.623878,41.199505,20.029067,20.009599,20.037149,0.49,0.40,0.02,0.00,0.09,0.0,0.0,0.00,74.56,5.12,4.51,0.00,15.81,0.0,0.0,0.00,21.168975,21.041027,1.264683,1.173199,5.029808,3.286035,3.977128,2.800918,20.046,1
3,327.30,95.9,23.2,21,2,7,1,1,7.4095,8.6034,20.247273,19.981341,9.0468,10.1334,19.674979,20.313131,1.158458,0.007002,8.6098,10.3467,20.480878,20.448428,3.6421,5.4166,20.249601,20.261903,2.5441,4.0777,19.653137,19.848343,22.342195,20.064190,20.024136,20.021322,0.32,0.38,0.06,0.03,0.09,0.0,0.0,0.12,40.37,4.00,9.78,9.80,12.84,0.0,0.0,23.22,20.819019,20.849014,2.363966,1.910184,3.384222,2.537386,1.431587,1.328347,20.074,1
5,298.29,83.2,22.2,22,0,4,1,1,9.5934,11.7111,20.286499,20.272651,3.2344,4.7739,19.999637,20.133996,21.080289,0.061452,8.7581,11.5765,20.309227,20.443822,2.5758,4.7816,20.035664,20.012962,2.6533,4.6505,20.991714,20.589871,154.163173,20.395606,20.754447,20.550477,0.44,0.39,0.11,0.00,0.06,0.0,0.0,0.00,64.42,4.73,21.45,0.00,9.39,0.0,0.0,0.00,22.966052,22.453152,3.400148,2.421052,3.300833,2.489302,0.970791,1.028191,20.072,1
7,219.31,80.0,22.6,15,1,4,3,1,7.5474,8.7409,20.369661,20.027436,5.7507,7.0497,19.768608,20.019556,1.972249,0.018433,6.0074,8.0148,21.097651,20.684857,4.9854,6.9405,19.752485,19.830325,2.3869,4.2534,20.195411,20.077647,53.661373,20.365355,20.492701,20.114749,0.39,0.46,0.00,0.04,0.11,0.0,0.0,0.00,60.25,5.98,0.00,14.62,19.16,0.0,0.0,0.00,21.312432,21.239897,1.204999,1.154787,2.516821,1.884328,2.088651,1.631753,20.048,1
8,262.33,80.6,22.3,18,2,4,3,1,6.2385,6.2385,20.106896,20.550457,7.5012,9.6012,20.113712,20.439456,5.613379,0.006140,8.4762,12.0991,20.052446,20.113814,2.9715,5.5357,19.246737,20.193336,2.6567,4.6231,19.980007,20.045099,333.703081,20.523869,20.974875,20.014273,0.41,0.44,0.06,0.03,0.06,0.0,0.0,0.00,59.52,5.38,12.20,12.22,10.68,0.0,0.0,0.00,20.831667,20.649763,2.852499,2.185650,3.190499,2.617097,1.118493,1.197400,20.038,1


In [3]:
# Check for NaN
df.isnull().values.any()

False

In [4]:
df.describe(include="all")

,Solubility_at_pH_7_4,MW,TPSA,XL,HAC,HBDC,HBAC,RBC,CBUC,MMX6,MMX,SX6,SX,MMY6,MMY,SY6,SY,MMX6_3D,MMX_3D,SX6_3D,SX_3D,MMY6_3D,MMY_3D,SY6_3D,SY_3D,MMZ6_3D,MMZ_3D,SZ6_3D,SZ_3D,XY_3D_volume,XZ_3D_volume,YZ_3D_volume,C_relative,H_relative,O_relative,N_relative,C,H,O,N,C_rel_2D,allAtoms_rel_2D,C_rel_XY_3D,allAtoms_rel_XY_3D,Similarity,Potency,Efficacy,Fit_HillSlope
count,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.0,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000,3885.000000
mean,24.361634,299.737155,79.608274,22.750612,20.892596,1.312158,4.261893,3.647277,1.0,9.593362,10.283051,20.025227,20.023212,8.159382,8.765068,19.947874,19.975820,10.223981,12.307840,20.009123,20.017082,4.618107,6.676298,20.002782,20.000748,2.296023,4.046446,19.993790,19.992605,18.128027,17.790673,17.791912,0.413014,0.425128,0.061913,0.073946,61.219765,5.403346,11.973594,12.503535,20.180952,20.172937,2.453540,1.933230,17.118336,31.479050,132.880253,2.413298
std,17.241094,45.157389,30.657511,1.124448,3.391663,0.885970,1.525376,1.723781,0.0,2.898028,2.738761,0.299497,0.267536,2.883387,2.706120,0.363526,0.297759,2.439487,2.363711,0.319172,0.259466,1.459867,1.391777,0.437487,0.289798,1.101190,1.294826,0.539671,0.341500,1.490453,1.691954,1.645702,0.045357,0.063565,0.038230,0.037101,9.024305,1.468958,7.028264,5.823354,0.540935,0.479724,1.028223,0.592331,0.266351,20.985018,29.285321,1.138034
min,0.100000,170.888044,3.200000,19.331142,11.536287,0.000000,1.000000,0.000000,1.0,1.963500,2.605200,19.129028,19.228516,0.750000,1.950000,18.842790,19.062327,3.287794,5.383595,19.064576,19.220588,0.394700,3.024900,18.642902,19.114697,0.000300,0.495030,18.287424,18.894876,11.044545,4.575188,8.169548,0.275566,0.240060,0.000000,0.020000,34.403135,1.105562,0.000000,2.950000,18.158090,18.158090,0.576067,0.798337,15.864833,0.281800,62.000400,0.500000
25%,6.300000,271.300000,58.900000,22.000000,19.000000,1.000000,3.000000,2.000000,1.0,7.541100,8.485000,19.833741,19.856221,5.989000,6.752400,19.717419,19.776418,8.530900,10.608300,19.814487,19.853848,3.483200,5.594100,19.739605,19.822663,1.590700,3.315000,19.732215,19.826098,17.207943,16.786987,16.845079,0.380000,0.380000,0.030000,0.050000,54.950000,4.350000,5.960000,8.480000,19.804198,19.829279,1.669687,1.480522,16.942392,11.220200,112.487000,1.538600
50%,25.800000,303.400000,78.400000,22.800000,21.000000,1.000000,4.000000,3.000000,1.0,9.147300,10.117200,20.004511,20.003985,7.923300,8.707400,19.956419,19.975859,10.103600,12.186500,20.003370,20.018825,4.532100,6.559000,20.000352,20.002806,2.417500,4.266100,19.998761,19.997944,18.177668,17.797996,17.817046,0.410000,0.430000,0.060000,0.070000,61.760000,5.300000,11.290000,11.960000,20.144619,20.142768,2.247541,1.845839,17.081229,35.481300,131.565000,2.252600
75%,39.700000,329.700000,98.500000,23.500000,23.000000,2.000000,5.000000,5.000000,1.0,11.454400,12.032200,20.198923,20.182187,9.916600,10.808300,20.185480,20.186501,11.755500,13.806000,20.199718,20.189823,5.591100,7.575600,20.270303,20.180943,2.900100,4.838700,20.259965,20.151872,19.117023,18.798135,18.767589,0.440000,0.470000,0.090000,0.090000,67.640000,6.390000,16.470000,16.040000,20.551068,20.521761,3.034758,2.286964,17.266632,44.668400,151.491000,3.065400
max,68.800000,433.128572,156.211157,25.887395,30.863674,3.646594,8.720489,9.242697,1.0,18.637373,18.854316,20.928048,20.822583,17.366064,17.396032,21.059688,20.892608,17.621519,19.505026,20.961890,20.806010,9.143978,10.877711,21.342798,20.878104,5.613939,7.834425,21.720358,21.105242,21.356788,23.156086,23.082981,0.548280,0.6134

[<a href="#content">Back to top</a>]

### Regression Machine Learning <a name="2"></a>

In [5]:
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


# Separate the training columns from the target column 'Potency'
X = df.drop(['MW' ], axis=1) 
y = df['MW'] 

# Split data set into train and test parts 
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20,
                                                    shuffle=True,
                                                    random_state=5) 
"""
# Add the custom compound to X_test
"""

# Standardise the train data
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

# Print the shape of each part
print("Shapes:")
print("X_train:", X_train.shape)
print("X_test: ", X_test.shape)
print("y_train:", y_train.shape)
print("y_test: ", y_test.shape)

Shapes:
X_train: (3096, 59)
X_test:  (775, 59)
y_train: (3096,)
y_test:  (775,)


In [6]:
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor

# Instantiate the algorithms that will be used, placing them in a dictionary 
regs = {"SVR":SVR(kernel='rbf'),
        "DecisionTree":DecisionTreeRegressor(), 
        "RandomForest":RandomForestRegressor(), 
        "GradientBoost":GradientBoostingRegressor(),}

In [8]:
# Instantiate and train the model
model = RandomForestRegressor(random_state=1).fit(X_train, y_train)

In [9]:
prediction = model.predict(X_test)
predicted = prediction[-1] 
predicted 

268.98070000000007

In [8]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import numpy as np

# Create statistics with the results of training with different algorithms
def model_fit(regs):
    fitted_model={}
    model_result = pd.DataFrame()
    for model_name, model in regs.items():
        model.fit(X_train_mms,y_train)
        fitted_model.update({model_name:model})
        model_dict = {}
        model_dict['1.Algorithm'] = model_name
        model_dict['2.RMSE_Train'] = round(np.sqrt (mean_squared_error(y_train, model.predict(X_train_mms))),2)
        model_dict['3.RMSE_Test'] = round( np.sqrt (mean_squared_error(y_test, model.predict(X_test_mms))),2)
        model_dict['4.MAE_Train'] = round(mean_absolute_error(y_train, model.predict(X_train_mms)),2)
        model_dict['5.MAE_Test'] = round(mean_absolute_error(y_test, model.predict(X_test_mms)),2)
        model_result = model_result._append(model_dict,ignore_index=True)
    return fitted_model, model_result

fitted_model, model_result = model_fit(regs)
model_result.sort_values(by=['5.MAE_Test'],ascending=True)

,1.Algorithm,2.RMSE_Train,3.RMSE_Test,4.MAE_Train,5.MAE_Test
2,RandomForest,7.70,20.10,6.39,16.78
0,SVR,20.43,20.30,16.99,16.98
3,GradientBoost,18.01,20.25,15.15,17.07
1,DecisionTree,0.00,29.31,0.00,23.42


[<a href="#content">Back to top</a>]

### Cross-validation <a name="3"></a>

In [ ]:
from sklearn.model_selection import cross_val_score
import numpy as np

# Create statistics with the results of cross-validation
def model_CV(regs):
    fitted_model={}
    model_cv_result = pd.DataFrame()
    for model_name, model in regs.items():
        fitted_model.update({model_name:model})
        scores = cross_val_score(model, X, y, cv=5,
                        scoring=('neg_mean_absolute_error'))
        scores = -scores
        model_dict = {}
        model_dict['1.Algorithm'] = model_name
        model_dict['2.CV_MAE'] = round(np.mean(scores), 2)
        model_dict['3.Sta Dev MAE'] = round(np.std(scores), 2)
        model_dict['4.List of MAE'] = np.round(scores, 2)
        model_cv_result = model_cv_result._append(model_dict,ignore_index=True)
    return fitted_model, model_cv_result

fitted_model, model_cv_result = model_CV(regs)
model_cv_result.sort_values(by=['2.CV_MAE'],ascending= True)

[<a href="#content">Back to top</a>]

### Feature importance <a name="4"></a>

In [ ]:
import matplotlib.pyplot as plt

# Instantiate and train the model
model = RandomForestRegressor(random_state=1).fit(X_train_mms, y_train)

# Define feature importance of RandomForestClassifier 
feat_imp = pd.Series(model.feature_importances_, index=X.columns)
feat_imp = feat_imp.nlargest(15).sort_values()

# Plot feature importance of RandomForestClassifier 
feat_imp.plot(kind="barh", title="Feature Importance ({:})".format(model))
plt.show()

In [ ]:
from sklearn.inspection import permutation_importance

# Create a list of the column names
feature_names = list(X)

# Convert the list into an array
features = np.array(feature_names)

# Calculate the permutation feature importance
perm_importance = permutation_importance(model, X_test_mms, y_test,
                                         scoring='neg_mean_absolute_error',
                                         random_state=0)

# Sort the result
sorted_idx = perm_importance.importances_mean.argsort()

# Plot the permutation feature importance
plt.figure(figsize=(10,20))
plt.barh(features[sorted_idx], perm_importance.importances_mean[sorted_idx])
plt.xlabel("Permutation Importance")

[<a href="#content">Back to top</a>]

### Comparison between different feature importance methods. Creation of a model only with solubility data <a name="5"></a>

In [ ]:

# import numpy as np
# from sklearn import preprocessing
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.ensemble import RandomForestClassifier

# from sklearn import metrics
# from sklearn.metrics import confusion_matrix
# from matplotlib import pyplot as plt

# import pandas as pd 
# from sklearn.model_selection import cross_validate

# # loading the dataset for the classification ML
# df = pd.read_csv('data_regression.csv', index_col=[0])

In [ ]:
import numpy as np
from sklearn import metrics

# Create a data frame with the features selected by the permutation_importance method
# # Freature importance
# df_features_importance_selected = df[['Solubility_at_pH_7_4',  # 18.54
#                                       'TPSA',                  # 18.13
#                                       'SX_3D',                 # 17.97
#                                       'XY_3D_volume',          # 17.82
#                                       'SX6_3D',                # 17.45
#                                       'SX',                    # 17.40
#                                       'Similarity',            # 17.67
#                                       'SX6',                   # 17.53
#                                       'MMX_3D',                # 17.35
#                                       'SY_3D',                 # 17.46
#                                       'XZ_3D_volume',          # 17.34
#                                       'SY6',                   # 17.36
#                                       'SY6_3D',                # 17.4
#                                       'MW',                    # 17.24
#                                       'YZ_3D_volume',          # 17.29
#                                       'Potency']]
# # Pernutation feature importance
df_permutation_selected = df[['Solubility_at_pH_7_4',            # 18.54
                              # 'allAtoms_rel_XY_3D',              # 18.48
                              # 'MMX',                             # 18.13
                              # 'MW',                              # 17.46
                              # 'TPSA',                            # 16.94                 
                              # 'MMY_3D',                          # 17.15
                              # 'MMX_3D',                          # 17.43
                              # 'MMZ_3D',                          # 17.02
                              # 'YZ_3D_volume',                    # 16.89
                              # 'HBDC',                            # 16.82
                              # 'C_rel_2D',                        # 16.85
                              # 'SZ6_3D',                          # 16.97
                              # 'MMX6_3D',                         # 16.85
                              # 'MMX6_3D',                         # 16.82
                              # 'MMY6_3D',                         # 16.67
                              # 'SY6',                             # 16.63
                              'Potency']]

# Separate columns for training from the target column
X = df_permutation_selected.drop(['Potency'], axis=1)
y = df_permutation_selected ['Potency']

# Split data in train and test parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=5) 

# Standardise the train data
X_train_mms = MinMaxScaler().fit_transform(X_train)
X_test_mms = MinMaxScaler().fit_transform(X_test)

# Instantiate and train a model
model = RandomForestRegressor(random_state=1).fit(X_train_mms, y_train)

# Predict 
pred = model.predict(X_test_mms)

print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))
mape = np.mean(np.abs((y_test - pred) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))

[<a href="#content">Back to top</a>]

### Hyperparameter tuning of the model with the solubility data only

In [ ]:
import pprint as pp

# Currentlly used parameters
pp.pprint(model.get_params())

In [ ]:
from sklearn.model_selection import GridSearchCV

# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 3)]

# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 3)]
max_depth.append(None)

# Minimum number of samples required to split a node
min_samples_split = [2, 5, 10]

# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]

# Method of selecting samples for training each tree
bootstrap = [True, False]

# Create the random grid
param_grid = {'n_estimators': n_estimators,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}
pp.pprint(param_grid)

In [ ]:
# Use the random grid to search for best hyperparameters
# Create the base model to tune
rfr = RandomForestRegressor()

# Random search of parameters, using 3 fold cross validation, 
# search across 100 different combinations, and use all available cores
grid_search_rfr= GridSearchCV(rfr, param_grid, cv = 5, verbose=2, n_jobs = -1)

# Fit the random search model
grid_search_rfr.fit(X_train_mms, y_train)

# Print
print('Optimal parameters: ', grid_search_rfr.best_params_)
print('Best score (on validation data): ', grid_search_rfr.best_score_)
print('Test set score: ', grid_search_rfr.score(X_test_mms, y_test))

In [ ]:
#Instantiate and train a model with the tuned hyperparameters
model = RandomForestRegressor(bootstrap=True, 
                              max_depth=10, 
                              min_samples_leaf=4,
                              min_samples_split=10,
                              n_estimators=2000,
                              random_state=1).fit(X_train_mms, y_train)

# Predict 
pred = model.predict(X_test_mms)

# Evaluate
print('Mean Absolute Error (MAE):', metrics.mean_absolute_error(y_test, pred))
print('Mean Squared Error (MSE):', metrics.mean_squared_error(y_test, pred))
print('Root Mean Squared Error (RMSE):', np.sqrt(metrics.mean_squared_error(y_test, pred)))
mape = np.mean(np.abs((y_test - pred) / np.abs(y_test)))
print('Mean Absolute Percentage Error (MAPE):', round(mape * 100, 2))
print('Accuracy:', round(100*(1 - mape), 2))

[<a href="#content">Back to top</a>]

### Demo with samples unseen by the model with the solubility data only <a name="7"></a>

In [ ]:
# Load data with the ten examples 
df_user_compounds = pd.read_csv('demo_FitHillSlope_Potency_Efficacy.csv', index_col=[0])

# Display the new demo data frame
df_user_compounds 

In [ ]:
# Create a data frame from the demo dataset only with the chosen above columns
df_user_compounds_with_target = df_user_compounds[['Solubility_at_pH_7_4',
                                                   'Potency']]

# Display the new demo data frame
df_user_compounds_with_target 

In [ ]:
# Remain only the data for training
df_user_compounds = df_user_compounds[['Solubility_at_pH_7_4',               # 1.22
                             # 'TPSA',                  # 1.22
                             #  'MMX'
                                      ]] 
# Display the new demo data frame
df_user_compounds 

In [ ]:
# Extract the min and max values
X_test.describe(include="all")

In [ ]:
# Create a dictionary with min and max values
dict = {'Solubility_at_pH_7_4':[0.1, 61.9]}

# Create a data frame with min and max values
df_minmax = pd.DataFrame(dict)

# Display the min_max data frame
df_minmax

In [ ]:
# Concatenate the mon_max data frame to the demo samples data frame 
df_user_compounds = pd.concat([df_user_compounds, df_minmax],ignore_index=True, sort=False)

# Display the resulting data frame 
df_user_compounds

In [ ]:
X_test.head()

In [ ]:
# Standardise the train columns of the demo data frame 
X_test_user = preprocessing.MinMaxScaler().fit_transform(df_user_compounds)

# Display the standardise data frame 
X_test_user

In [ ]:
# Predict using demo samples 
user_input_predictions = model.predict(X_test_user)

# Drop the min and max samples
user_input_predictions = np.round(np.r_[user_input_predictions[:-2]], 2)

# Display the prediction for the demo samples
user_input_predictions

In [ ]:
# Create a variable with the test data
y_test = round(df_user_compounds_with_target["Potency"], 2)

In [ ]:
# Create a data frame with the test values 
data_verify=pd.DataFrame(y_test.tolist(),columns=["Real Values"])

In [ ]:
# Create a data frame with the values predicted 
data_predicted=pd.DataFrame(user_input_predictions.tolist(),columns=["Predicted Values"])

In [ ]:
# Concatenate the data frames with the test and the values predicted
final_output=pd.concat([data_verify,data_predicted],axis=1)

# Create column with the difference between the test and prediction values
final_output["Difference"]=final_output["Real Values"]-final_output["Predicted Values"]

# Display the resulted data frame 
final_output

[<a href="#content">Back to top</a>]

### NN with PyTorch. All features. Optuna hyperparameter tuning <a name="8"></a>

In [ ]:
# pip install optuna
# https://github.com/optuna/optuna-examples/blob/main/pytorch/pytorch_simple.py

In [ ]:
import os

import optuna
from optuna.trial import TrialState
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.utils.data

In [ ]:
import pandas as pd
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from sklearn import metrics
import matplotlib.pyplot as plt

In [ ]:
# load data
df = pd.read_csv('data_regression.csv', index_col=[0])

# Separate the column with the targets from the rest of the columns of the data set
X = df.drop(['Fit_HillSlope', "Potency", "Efficacy" ], axis=1).to_numpy()  
y = df['Potency'].to_numpy()

# Split data into train and test parts in porportion 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20,
                                                    shuffle=True,
                                                    random_state=5) 

# Standardizing data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Convert data into tenzors
X_train = torch.Tensor(X_train).float()
y_train = torch.Tensor(y_train).float().reshape(-1, 1)

X_test = torch.Tensor(X_test).float()
y_test = torch.Tensor(y_test).float().reshape(-1, 1)

In [ ]:
DEVICE = torch.device("cpu")
BATCHSIZE = 128
TARGETS = 1
DIR = os.getcwd()
EPOCHS = 10
N_TRAIN_EXAMPLES = BATCHSIZE * 30
N_VALID_EXAMPLES = BATCHSIZE * 10

In [ ]:
X.shape[1]

In [ ]:
# Function to ptimize the number of layers, hidden units and dropout ratio in each layer.
def define_model(trial):
    n_layers = trial.suggest_int("n_layers", 1, 3)  # name, low, high,
    layers = []

    in_features = X.shape[1]
    for i in range(n_layers):
        out_features = trial.suggest_int("n_units_l{}".format(i), 4, 128)
        layers.append(nn.Linear(in_features, out_features))
        layers.append(nn.ReLU())
        p = trial.suggest_float("dropout_l{}".format(i), 0.2, 0.5)
        layers.append(nn.Dropout(p))

        in_features = out_features
    layers.append(nn.Linear(in_features, TARGETS))
    layers.append(nn.LogSoftmax(dim=1))

    return nn.Sequential(*layers)

In [ ]:
from torch.utils.data import DataLoader, TensorDataset

# Instantiate tensor datasets in PyTorch
dataset_train = TensorDataset(X_train, y_train)
dataset_test = TensorDataset(X_test, y_test)

In [ ]:
# Tuning
def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the dataset.
    train_loader = DataLoader(dataset_train, 
                              batch_size=BATCHSIZE,
                              shuffle=True)
    valid_loader = DataLoader(dataset_test, 
                              batch_size=BATCHSIZE,
                              shuffle=True)

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.mse_loss(output, target)   # https://pytorch.org/docs/stable/nn.functional.html
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[<a href="#content">Back to top</a>]

### Tuned NN model <a name="9"></a>
https://www.youtube.com/watch?v=Ilr_V2fka8s

In [ ]:
# Create a structure of NN model with the suggested hyperparameters 
class Net(nn.Module):
    def __init__(self, in_count, out_count):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_count, 113)
        self.dropout = nn.Dropout(0.22199392333988044)
        self.fc2 = nn.Linear(113, 123)
        self.dropout = nn.Dropout(0.2715937718221687)
        self.fc3 = nn.Linear(123, out_count)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

In [ ]:
# Create and transfer the model to the device
model = Net(X.shape[1],1).to(DEVICE)

In [ ]:
# Define the loss function for regression
loss_fn = nn.MSELoss()

In [ ]:
# Define the optimizer 
optimizer =  torch.optim.SGD(model.parameters(), lr=0.000001)

In [ ]:
# Earlly Stopping
class EarllyStopping():
    def __init__(self, patience=5, min_delta=1e-2, restore_best_weights=True, best_model = None, best_loss = None):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = best_model
        self.best_loss = best_loss
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model)
        elif self.best_loss -val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model.load_state_dict(model.state_dict())
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            if self.counter >= self.patience: 
                self.status = f"Stopped on {self.counter}"
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model.state_dict())
                return True
        self.status = f"{self.counter}/{self.patience}"
        return False 

es = EarllyStopping()

In [ ]:
import copy
import tqdm

# Run the training 
epoch = 0
done = False

while epoch<1000 and not done:
    epoch +=1
    steps = list(enumerate(dataset_train))
    pbar = tqdm.tqdm(steps)
    model.train()
    for i, (x_batch, y_batch) in pbar:
        y_batch_pred = model(x_batch.to(DEVICE)).flatten()
        loss = loss_fn(y_batch_pred, y_batch.to(DEVICE))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        loss, current = loss.item(), (i + 1)* len(x_batch)
        if i == len(steps)-1:
            model.eval()
            pred = model(X_test).flatten()
            vloss = loss_fn(pred, y_test)
            if es(model, vloss): done = True
            pbar.set_description(f"Epoch: {epoch}, tloss: {loss}, vloss:{vloss:>7f}, EStop:[{es.status}]")
        else:
            pbar.set_description(f"Epoch: {epoch}, tloss: {loss}")

In [ ]:
# Predict 
pred = model(X_test)

# Measure MSE error 
score = metrics.mean_squared_error(pred.cpu().detach(), y_test.cpu())
print ("Final score (MSE): {}". format(score))

In [ ]:
# Measure RMSE error
score = np.sqrt(metrics.mean_squared_error(pred.cpu().detach(), y_test.cpu().detach()))
print ("Final score (RMSE): {}". format(score))

In [ ]:
# Regression chart 
def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred':pred, 'y': y.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='Expected')
    plt.plot(t['pred'].tolist(), label='Predicted')
    plt.legend()
    plt.show()

# Plot the chart 
chart_regression(pred.flatten().cpu().detach(), y_test.cpu().detach())
    

In [ ]:
# Create a data frame with the test values 
expected = pd.DataFrame(y_test.tolist(),columns=["Expected"])

In [ ]:
# Create a data frame with the predicted values 
predicted=pd.DataFrame(pred.tolist(),columns=["Predicted"])

In [ ]:
# Concatenate the data frames with the test and the poredicted values
final_output=pd.concat([expected, predicted],axis=1)

# Create column with the difference between the test and prediction values
final_output['Difference']=final_output['Expected']-final_output['Predicted']

# Display the resulted data frame 
final_output.head(10)

[<a href="#content">Back to top</a>]

### NN only with the solubility data. Optuna hyperparameter tuning <a name="10"></a>

In [ ]:
# Create a data frame with the considered five features
df = df[['Solubility_at_pH_7_4',
         'Potency']]

# Separate the column with the targets from the rest of the columns of the data set
X = df.drop(['Potency'], axis=1).to_numpy()  
y = df['Potency'].to_numpy()

# Split data into train and test parts in porportion 80/20
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.20,
                                                    shuffle=True,
                                                    random_state=5) 

# Standardizing data
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.fit_transform(X_test)

# Convert data into tenzors
X_train = torch.Tensor(X_train).float()
y_train = torch.Tensor(y_train).float().reshape(-1, 1)

X_test = torch.Tensor(X_test).float()
y_test = torch.Tensor(y_test).float().reshape(-1, 1)

In [ ]:
# Instantiate tensor datasets in PyTorch
dataset_train = TensorDataset(X_train, y_train)
dataset_test = TensorDataset(X_test, y_test)

In [ ]:
# Do tuning
def objective(trial):
    # Generate the model.
    model = define_model(trial).to(DEVICE)

    # Generate the optimizers.
    optimizer_name = trial.suggest_categorical("optimizer", ["Adam", "RMSprop", "SGD"])
    lr = trial.suggest_float("lr", 1e-5, 1e-1, log=True)
    optimizer = getattr(optim, optimizer_name)(model.parameters(), lr=lr)

    # Get the FashionMNIST dataset.
    # train_loader, valid_loader = get_mnist()
    train_loader = DataLoader(dataset_train, 
                              batch_size=BATCHSIZE,
                              shuffle=True)
    valid_loader = DataLoader(dataset_test, 
                              batch_size=BATCHSIZE,
                              shuffle=True)

    # Training of the model.
    for epoch in range(EPOCHS):
        model.train()
        for batch_idx, (data, target) in enumerate(train_loader):
            # Limiting training data for faster epochs.
            if batch_idx * BATCHSIZE >= N_TRAIN_EXAMPLES:
                break

            data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)

            optimizer.zero_grad()
            output = model(data)
            loss = F.mse_loss(output, target)   # https://pytorch.org/docs/stable/nn.functional.html
            loss.backward()
            optimizer.step()

        # Validation of the model.
        model.eval()
        correct = 0
        with torch.no_grad():
            for batch_idx, (data, target) in enumerate(valid_loader):
                # Limiting validation data.
                if batch_idx * BATCHSIZE >= N_VALID_EXAMPLES:
                    break
                data, target = data.view(data.size(0), -1).to(DEVICE), target.to(DEVICE)
                output = model(data)
                # Get the index of the max log-probability.
                pred = output.argmax(dim=1, keepdim=True)
                correct += pred.eq(target.view_as(pred)).sum().item()

        accuracy = correct / min(len(valid_loader.dataset), N_VALID_EXAMPLES)

        trial.report(accuracy, epoch)

        # Handle pruning based on the intermediate value.
        if trial.should_prune():
            raise optuna.exceptions.TrialPruned()

    return accuracy

if __name__ == "__main__":
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, timeout=600)

    pruned_trials = study.get_trials(deepcopy=False, states=[TrialState.PRUNED])
    complete_trials = study.get_trials(deepcopy=False, states=[TrialState.COMPLETE])

    print("Study statistics: ")
    print("  Number of finished trials: ", len(study.trials))
    print("  Number of pruned trials: ", len(pruned_trials))
    print("  Number of complete trials: ", len(complete_trials))

    print("Best trial:")
    trial = study.best_trial

    print("  Value: ", trial.value)

    print("  Params: ")
    for key, value in trial.params.items():
        print("    {}: {}".format(key, value))

[<a href="#content">Back to top</a>]

### Tuned NN model with the solubility data <a name="11"></a>

In [ ]:
# Create a structure of NN model with the suggested hyperparameters 
class Net(nn.Module):
    ef __init__(self, in_count, out_count):
        super(Net, self).__init__()
        self.dropout = nn.Dropout(0.38391677264607094)
        self.fc1 = nn.Linear(in_count, 13)
        self.dropout = nn.Dropout(0.48142911797548454)
        self.fc2 = nn.Linear(13, 66)
        self.fc3 = nn.Linear(66, out_count)

    def forward(self, x):
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return self.fc3(x)

In [ ]:
# Create and transfer the model to the device
model = Net(X.shape[1],1).to(DEVICE)

In [ ]:
# Define the loss function for regression
loss_fn = nn.MSELoss()

In [ ]:
# Define the optimizer 
optimizer =  torch.optim.RMSprop(model.parameters(), lr=0.021957450493775982)

In [ ]:
# Earlly Stopping
class EarllyStopping():
    def __init__(self, patience=5, min_delta=1e-2, restore_best_weights=True, best_model = None, best_loss = None):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = best_model
        self.best_loss = best_loss
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss == None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model)
        elif self.best_loss -val_loss > self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.best_model.load_state_dict(model.state_dict())
        elif self.best_loss - val_loss < self.min_delta:
            self.counter += 1
            if self.counter >= self.patience: 
                self.status = f"Stopped on {self.counter}"
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model.state_dict())
                return True
        self.status = f"{self.counter}/{self.patience}"
        return False 

es = EarllyStopping()

In [ ]:
import copy
import tqdm

# Run the training 
epoch = 0
done = False

while epoch<1000 and not done:
    epoch +=1
    steps = list(enumerate(dataset_train))
    pbar = tqdm.tqdm(steps)
    model.train()
    for i, (x_batch, y_batch) in pbar:
        y_batch_pred = model(x_batch.to(DEVICE)).flatten()
        loss = loss_fn(y_batch_pred, y_batch.to(DEVICE))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
        loss, current = loss.item(), (i + 1)* len(x_batch)
        if i == len(steps)-1:
            model.eval()
            pred = model(X_test).flatten()
            vloss = loss_fn(pred, y_test)
            if es(model, vloss): done = True
            pbar.set_description(f"Epoch: {epoch}, tloss: {loss}, vloss:{vloss:>7f}, EStop:[{es.status}]")
        else:
            pbar.set_description(f"Epoch: {epoch}, tloss: {loss}")

In [ ]:
# Predict 
pred = model(X_test)

# Measure MSE error 
score = metrics.mean_squared_error(pred.cpu().detach(), y_test.cpu())
print ("Final score (MSE): {}". format(score))

In [ ]:
# Measure RMSE error
score = np.sqrt(metrics.mean_squared_error(pred.cpu().detach(), y_test.cpu().detach()))
print ("Final score (RMSE): {}". format(score))

In [ ]:
# Regression chart 
def chart_regression(pred, y, sort=True):
    t = pd.DataFrame({'pred':pred, 'y': y.flatten()})
    if sort:
        t.sort_values(by=['y'], inplace=True)
    plt.plot(t['y'].tolist(), label='Expected')
    plt.plot(t['pred'].tolist(), label='Predicted')
    plt.legend()
    plt.show()

# Plot the chart 
chart_regression(pred.flatten().cpu().detach(), y_test.cpu().detach())

In [ ]:
# Create a data frame with the test values 
expected = pd.DataFrame(y_test.tolist(),columns=["Expected"])

In [ ]:
# Create a data frame with the predicted values 
predicted=pd.DataFrame(pred.tolist(),columns=["Predicted"])

In [ ]:
# Concatenate the data frames with the test and the poredicted values
final_output=pd.concat([expected, predicted],axis=1)

# Create column with the difference between the test and prediction values
final_output['Difference']=final_output['Expected']-final_output['Predicted']

# Display the resulted data frame 
final_output.head(10)

[<a href="#content">Back to top</a>]